In [1]:
import pandas as pd

In [2]:
transactions = pd.read_csv("old/sidooh_transactions.csv")
at_airtime_requests = pd.read_csv("old/sidooh_airtime_requests.csv")
at_airtime_responses = pd.read_csv("old/sidooh_airtime_responses.csv")
cashbacks = pd.read_csv("old/sidooh_earnings.csv")

airtime_accounts = pd.read_csv("old/sidooh_airtime_accounts.csv")
utility_accounts = pd.read_csv("old/sidooh_utility_accounts.csv", dtype={'utility_number': str, })

subscriptions = pd.read_csv("old/sidooh_subscriptions.csv")

In [3]:
transactions_new = transactions.copy()
at_airtime_requests_new = at_airtime_requests.copy()
at_airtime_responses_new = at_airtime_responses.copy()
cashbacks_new = cashbacks.copy()

airtime_accounts_new = airtime_accounts.copy()
utility_accounts_new = utility_accounts.copy()
subscriptions_new = subscriptions.copy()

In [4]:
# transactions_new = transactions.copy()

# No need we still have pending transactions from previous year that need to be resolved
# dateFrom = pd.to_datetime('2022')

# transactions_new = transactions[pd.to_datetime(transactions['created_at']) > dateFrom]

transactions_new['initiator'] = 'CONSUMER'
transactions_new[['description', 'destination']] = transactions.description.str.split(' - ', expand=True)

def statusTransform(val):
    val = val.upper()
    
    if (val == "REIMBURSED"):
        val = "REFUNDED"
        
    if (val == "SUCCESS"):
        val = "COMPLETED"
    
    return val


transactions_new['status'] = transactions['status'].apply(statusTransform)
transactions_new

,id,amount,status,type,description,created_at,updated_at,account_id,product_id,initiator,destination
0,1,20,COMPLETED,PAYMENT,Airtime Purchase,2021-02-01 17:10:23,2021-02-01 17:14:00,1,1,CONSUMER,254714611696
1,2,50,COMPLETED,PAYMENT,Airtime Purchase,2021-02-01 17:11:04,2021-02-01 17:11:50,2,1,CONSUMER,254711414987
2,3,475,COMPLETED,PAYMENT,Subscription Purchase,2021-02-01 17:17:15,2021-02-01 17:17:15,1,2,CONSUMER,None
3,4,8775,FAILED,PAYMENT,Subscription Purchase,2021-02-01 17:20:44,2021-02-01 17:20:44,2,2,CONSUMER,None
4,5,20,FAILED,PAYMENT,Airtime Purchase,2021-02-01 17:22:49,2021-02-01 17:22:49,3,1,CONSUMER,254733643843
...,...,...,...,...,...,...,...,...,...,...,...
12992,12993,100,COMPLETED,PAYMENT,Airtime Purchase,2022-07-21 18:26:49,2022-07-21 18:27:05,128,1,CONSUMER,254723604233
12993,12994,50,COMPLETED,PAYMENT,Airtime Purchase,2022-07-21 18:51:13,2022-07-21 18:51:27,651,1,CONSUMER,254791570802
12994,12995,50,FAILED,PAYMENT,Airtime Purchase,2022-07-21 18:53:08,2022-07-21 18:53:19,651,1,CONSUMER,254780130388
12995,12996,80,COMPLETED,PAYMENT,Airtime Purchase,2022-07-21 18:55:05,2022-07-21 18:55:22,651,1,CONSUMER,254791570802


In [5]:
at_airtime_requests_new['amount'] = at_airtime_requests['totalAmount'].apply(lambda val: val.split(' ')[1] if len(val.split(' ')) == 2 else val)
at_airtime_requests_new['discount'] = at_airtime_requests['totalDiscount'].apply(lambda val: val.split(' ')[1] if len(val.split(' ')) == 2 else val)
del at_airtime_requests_new['totalAmount']
del at_airtime_requests_new['totalDiscount']

In [6]:
at_airtime_requests_new

,id,errorMessage,numSent,transaction_id,created_at,updated_at,amount,discount
0,1,None,1,2,2021-02-01 17:11:48,2021-02-01 17:11:48,50.0000,2.0000
1,2,None,1,1,2021-02-01 17:13:57,2021-02-01 17:13:57,20.0000,0.8000
2,3,None,1,6,2021-02-01 17:28:36,2021-02-01 17:28:36,20.0000,0.8000
3,4,None,1,8,2021-02-01 17:50:34,2021-02-01 17:50:34,50.0000,2.0000
4,5,None,1,9,2021-02-01 18:17:24,2021-02-01 18:17:24,20.0000,0.8000
...,...,...,...,...,...,...,...,...
7003,7004,None,0,8181,2021-11-06 11:31:49,2021-11-06 11:31:49,0,0
7004,7005,None,0,8184,2021-11-06 11:31:50,2021-11-06 11:31:50,0,0
7005,7006,None,0,8186,2021-11-06 11:31:51,2021-11-06 11:31:51,0,0
7006,7007,None,0,8211,2021-11-06 18:34:33,2021-11-06 18:34:33,0,0


In [7]:
at_airtime_responses_new['amount'] = at_airtime_responses['amount'].apply(lambda val: val.split(' ')[1] if len(val.split(' ')) == 2 else val)
at_airtime_responses_new['discount'] = at_airtime_responses['discount'].apply(lambda val: val.split(' ')[1] if len(val.split(' ')) == 2 else val)

# Don't think it is so necessary
# at_airtime_responses_new['status'] = at_airtime_responses['status'].apply(lambda val: val.upper())

In [8]:
at_airtime_responses_new

,id,phoneNumber,errorMessage,amount,status,requestID,discount,airtime_request_id,created_at,updated_at
0,1,254711414987,None,50.0000,Success,ATQid_cb7a75800e11ba3231b9599e0ecf64db,2.0000,1,2021-02-01 17:11:48,2021-02-01 17:11:49
1,2,254714611696,None,20.0000,Success,ATQid_09165c68ba96cd8ec6f68ef96683a9e2,0.8000,2,2021-02-01 17:13:57,2021-02-01 17:13:59
2,3,254733643843,None,20.0000,Success,ATQid_3ea0f33ef0d04255030f634fcc0dc483,0.8000,3,2021-02-01 17:28:36,2021-02-01 17:28:37
3,4,254714611696,None,50.0000,Success,ATQid_385061e833dd49fe8382276c38007241,2.0000,4,2021-02-01 17:50:34,2021-02-01 17:50:36
4,5,254725839446,None,20.0000,Success,ATQid_a6783f6a04c058748978634948ae5a28,0.8000,5,2021-02-01 18:17:24,2021-02-01 18:17:26
...,...,...,...,...,...,...,...,...,...,...
6937,6938,254799180654,Insufficient Credit,100.0000,Failed,None,0,7004,2021-11-06 11:31:49,2021-11-06 11:31:49
6938,6939,254714611696,Insufficient Credit,20.0000,Failed,None,0,7005,2021-11-06 11:31:50,2021-11-06 11:31:50
6939,6940,254780490721,Insufficient Credit,30.0000,Failed,None,0,7006,2021-11-06 11:31:51,2021-11-06 11:31:51
6940,6941,254742666091,Insufficient Credit,50.0000,Failed,None,0,7007,2021-11-06 18:34:33,2021-11-06 18:34:33


In [9]:
cashbacks_new['amount'] = cashbacks['earnings']
del cashbacks_new['earnings']
del cashbacks_new['aggregate_transactions']

def typeTransform(val):
    val = val.upper()
    
    if (val == "REFERRAL"):
        val = "INVITE"
        
    return val

cashbacks_new['type'] = cashbacks['type'].apply(typeTransform)
cashbacks_new

,id,created_at,updated_at,account_id,transaction_id,type,amount
0,1,2021-02-01 17:11:50,2021-02-01 17:11:50,2.0,2,SELF,0.25
1,2,2021-02-01 17:11:50,2021-02-01 17:11:50,NaN,2,SYSTEM,0.50
2,3,2021-02-01 17:11:50,2021-02-01 17:11:50,NaN,2,SYSTEM,1.25
3,4,2021-02-01 17:14:00,2021-02-01 17:14:00,1.0,1,SELF,0.10
4,5,2021-02-01 17:14:00,2021-02-01 17:14:00,NaN,1,SYSTEM,0.20
...,...,...,...,...,...,...,...
40432,40433,2022-07-21 18:55:22,2022-07-21 18:55:22,NaN,12996,SYSTEM,1.92
40433,40434,2022-07-21 18:55:22,2022-07-21 18:55:22,NaN,12996,SYSTEM,2.40
40434,40435,2022-07-21 18:58:30,2022-07-21 18:58:30,651.0,12997,SELF,0.30
40435,40436,2022-07-21 18:58:30,2022-07-21 18:58:30,NaN,12997,SYSTEM,1.20


In [10]:
dateFrom = pd.to_datetime('2022')

cashbacks_new = cashbacks_new[pd.to_datetime(cashbacks_new['created_at']) > dateFrom]

cashbacks_new

,id,created_at,updated_at,account_id,transaction_id,type,amount
30494,30495,2022-01-01 06:58:30,2022-01-01 06:58:30,597.0,9638,SELF,0.18
30495,30496,2022-01-01 06:58:30,2022-01-01 06:58:30,6.0,9638,INVITE,0.18
30496,30497,2022-01-01 06:58:30,2022-01-01 06:58:30,1.0,9638,INVITE,0.18
30497,30498,2022-01-01 06:58:30,2022-01-01 06:58:30,NaN,9638,SYSTEM,0.72
30498,30499,2022-01-01 06:58:30,2022-01-01 06:58:30,NaN,9638,SYSTEM,0.54
...,...,...,...,...,...,...,...
40432,40433,2022-07-21 18:55:22,2022-07-21 18:55:22,NaN,12996,SYSTEM,1.92
40433,40434,2022-07-21 18:55:22,2022-07-21 18:55:22,NaN,12996,SYSTEM,2.40
40434,40435,2022-07-21 18:58:30,2022-07-21 18:58:30,651.0,12997,SELF,0.30
40435,40436,2022-07-21 18:58:30,2022-07-21 18:58:30,NaN,12997,SYSTEM,1.20


In [11]:
airtime_accs_mask = utility_accounts_new['provider'].isin(['SAFARICOM'])
airt_accs = utility_accounts_new[airtime_accs_mask]
utility_accounts_new = utility_accounts_new[~airtime_accs_mask]

del utility_accounts_new['id']
utility_accounts_new

,provider,utility_number,created_at,updated_at,account_id
0,KPLC_PREPAID,37170026761,NaN,NaN,4
1,KPLC_PREPAID,14105984695,NaN,NaN,9
2,KPLC_POSTPAID,24193922,NaN,NaN,717
3,NAIROBI_WTR,1739412,NaN,NaN,717
4,NAIROBI_WTR,5205829,NaN,NaN,1
5,NAIROBI_WTR,1739412,2021-11-12 20:16:14,2021-11-12 20:16:14,1
6,KPLC_PREPAID,37170026761,2021-11-18 07:15:19,2021-11-18 07:15:19,2
7,KPLC_PREPAID,37193810837,2021-11-24 09:06:11,2021-11-24 09:06:11,113
8,KPLC_PREPAID,54600122953,2021-11-26 18:47:27,2021-11-26 18:47:27,179
11,KPLC_POSTPAID,24193781,2021-11-27 01:49:37,2021-11-27 01:49:37,1


In [12]:
floating_airt_accs = airt_accs.copy()
accounts = pd.read_csv("old/sidooh_accounts.csv", dtype={'phone': str, 'pin': str, 'user_id': str, 'referrer_id': str},)

floating_airt_accs['phone'] = floating_airt_accs['utility_number']
floating_airt_accs = floating_airt_accs.merge(accounts.drop_duplicates(), on=['phone'], 
                   how='left', indicator=True)

floating_airt_accs = floating_airt_accs[floating_airt_accs['_merge'] == 'left_only']

selector_d = {
    'provider': 'provider',
    'utility_number': 'account_number',
    'created_at_x': 'created_at',
    'updated_at_x': 'updated_at',
    'account_id': 'account_id'
}

floating_airt_accs = floating_airt_accs.rename(columns=selector_d)[selector_d.values()]
floating_airt_accs

,provider,account_number,created_at,updated_at,account_id
31,SAFARICOM,254701713462,2021-11-29 18:05:31,2021-11-29 18:05:31,651
32,SAFARICOM,254719184630,2021-11-30 05:49:00,2021-11-30 05:49:00,654
52,SAFARICOM,254719183707,2021-12-03 21:02:45,2021-12-03 21:02:45,732
57,SAFARICOM,254729962750,2021-12-04 13:33:23,2021-12-04 13:33:23,708
60,SAFARICOM,254757775132,2021-12-05 12:52:35,2021-12-05 12:52:35,732
63,SAFARICOM,254723226317,2021-12-06 11:49:26,2021-12-06 11:49:26,708
65,SAFARICOM,254724787768,2021-12-07 18:11:51,2021-12-07 18:11:51,732
68,SAFARICOM,254720434274,2021-12-09 09:15:03,2021-12-09 09:15:03,753
75,SAFARICOM,254725351760,2021-12-12 14:26:45,2021-12-12 14:26:45,244
76,SAFARICOM,254723420623,2021-12-12 14:33:50,2021-12-12 14:33:50,244


In [13]:
airtime_accounts_new

,id,provider,airtime_number,created_at,updated_at,account_id
0,1,AIRTEL,254780611696,2021-11-27 01:40:57,2021-11-27 01:40:57,1
1,2,AIRTEL,254736388405,2021-11-27 01:56:26,2021-11-27 01:56:26,6
2,3,TELKOM,254773394505,2021-11-27 06:26:45,2021-11-27 06:26:45,274
3,4,TELKOM,254777535195,2021-11-27 07:50:08,2021-11-27 07:50:08,685
4,5,AIRTEL,254739630484,2021-11-27 07:56:41,2021-11-27 07:56:41,735
...,...,...,...,...,...,...
56,57,AIRTEL,254735583726,2022-01-06 11:49:20,2022-01-06 11:49:20,228
57,58,TELKOM,254776370364,2022-01-10 18:58:31,2022-01-10 18:58:31,514
58,59,AIRTEL,254734949783,2022-01-11 04:24:55,2022-01-11 04:24:55,364
59,60,AIRTEL,254736866553,2022-01-12 08:15:04,2022-01-12 08:15:04,627


In [14]:
del airtime_accounts_new['id']
airtime_accounts_new['account_number'] = airtime_accounts_new['airtime_number']
del airtime_accounts_new['airtime_number']

airtime_accounts_new = pd.concat([airtime_accounts_new, floating_airt_accs])
airtime_accounts_new

,provider,created_at,updated_at,account_id,account_number
0,AIRTEL,2021-11-27 01:40:57,2021-11-27 01:40:57,1,254780611696
1,AIRTEL,2021-11-27 01:56:26,2021-11-27 01:56:26,6,254736388405
2,TELKOM,2021-11-27 06:26:45,2021-11-27 06:26:45,274,254773394505
3,TELKOM,2021-11-27 07:50:08,2021-11-27 07:50:08,685,254777535195
4,AIRTEL,2021-11-27 07:56:41,2021-11-27 07:56:41,735,254739630484
...,...,...,...,...,...
87,SAFARICOM,2021-12-20 18:22:08,2021-12-20 18:22:08,232,254722889183
89,SAFARICOM,2021-12-23 21:33:01,2021-12-23 21:33:01,693,254728956087
90,SAFARICOM,2021-12-27 05:11:19,2021-12-27 05:11:19,654,254724205203
95,SAFARICOM,2022-01-03 08:53:18,2022-01-03 08:53:18,651,254757843330


In [15]:
earning_accounts = cashbacks_new.groupby(['account_id','type']).sum().reset_index()[['account_id', 'type','amount']]
earning_accounts

,account_id,type,amount
0,1.0,INVITE,239.138
1,1.0,SELF,46.640
2,2.0,INVITE,267.355
3,2.0,SELF,8.969
4,4.0,INVITE,107.814
...,...,...,...
137,816.0,SELF,0.180
138,817.0,SELF,7.500
139,820.0,SELF,1.200
140,821.0,SELF,0.360


In [16]:
earning_accounts = earning_accounts.pivot_table('amount', 'account_id', 'type', fill_value=0)
earning_accounts.reset_index( inplace=True )

earning_accounts['self_amount'] = earning_accounts['SELF']
earning_accounts['invite_amount'] = earning_accounts['INVITE']
earning_accounts['type'] = 'PURCHASES'
earning_accounts[['created_at', 'updated_at']] = pd.to_datetime('now')

del earning_accounts['SELF']
del earning_accounts['INVITE']

In [17]:
earning_accounts

type,account_id,self_amount,invite_amount,type,created_at,updated_at
0,1.0,46.640,239.138,PURCHASES,2022-07-27 18:26:36.941369,2022-07-27 18:26:36.941369
1,2.0,8.969,267.355,PURCHASES,2022-07-27 18:26:36.941369,2022-07-27 18:26:36.941369
2,4.0,27.278,107.814,PURCHASES,2022-07-27 18:26:36.941369,2022-07-27 18:26:36.941369
3,5.0,2.640,0.000,PURCHASES,2022-07-27 18:26:36.941369,2022-07-27 18:26:36.941369
4,6.0,36.654,166.851,PURCHASES,2022-07-27 18:26:36.941369,2022-07-27 18:26:36.941369
...,...,...,...,...,...,...
121,816.0,0.180,0.000,PURCHASES,2022-07-27 18:26:36.941369,2022-07-27 18:26:36.941369
122,817.0,7.500,0.000,PURCHASES,2022-07-27 18:26:36.941369,2022-07-27 18:26:36.941369
123,820.0,1.200,0.000,PURCHASES,2022-07-27 18:26:36.941369,2022-07-27 18:26:36.941369
124,821.0,0.360,0.000,PURCHASES,2022-07-27 18:26:36.941369,2022-07-27 18:26:36.941369


In [18]:
subscriptions_new

,id,amount,active,created_at,updated_at,account_id,subscription_type_id
0,1,475,1,2021-02-01 17:17:15,2021-02-01 17:17:15,1,1
1,2,18,0,2021-02-01 17:32:56,2021-02-01 17:32:56,2,4
2,3,18,0,2021-02-01 17:33:26,2021-02-01 17:33:26,2,4
3,4,18,0,2021-02-01 18:45:24,2022-02-02 00:00:02,1,4
4,5,18,0,2021-02-01 18:45:53,2021-02-01 18:45:53,1,4
5,6,18,1,2021-03-08 16:08:49,2021-03-08 16:08:49,2,4
6,7,18,0,2021-03-09 07:15:17,2021-03-09 07:15:17,4,4
7,8,365,0,2021-09-17 03:37:43,2021-10-18 00:00:15,6,5
8,9,365,0,2021-09-20 07:51:20,2021-10-21 00:00:00,182,5
9,10,365,0,2021-09-22 06:27:41,2021-10-23 00:00:16,242,5


In [19]:
subscriptions_new['start_date'] = subscriptions_new['created_at']
subscriptions_new['end_date'] = pd.to_datetime(subscriptions_new['created_at']) + pd.DateOffset(months=1)
subscriptions_new['subscription_type_id'] = 1
subscriptions_new['status'] = 'EXPIRED'
del subscriptions_new['amount']
del subscriptions_new['active']

subscriptions_new

,id,created_at,updated_at,account_id,subscription_type_id,start_date,end_date,status
0,1,2021-02-01 17:17:15,2021-02-01 17:17:15,1,1,2021-02-01 17:17:15,2021-03-01 17:17:15,EXPIRED
1,2,2021-02-01 17:32:56,2021-02-01 17:32:56,2,1,2021-02-01 17:32:56,2021-03-01 17:32:56,EXPIRED
2,3,2021-02-01 17:33:26,2021-02-01 17:33:26,2,1,2021-02-01 17:33:26,2021-03-01 17:33:26,EXPIRED
3,4,2021-02-01 18:45:24,2022-02-02 00:00:02,1,1,2021-02-01 18:45:24,2021-03-01 18:45:24,EXPIRED
4,5,2021-02-01 18:45:53,2021-02-01 18:45:53,1,1,2021-02-01 18:45:53,2021-03-01 18:45:53,EXPIRED
5,6,2021-03-08 16:08:49,2021-03-08 16:08:49,2,1,2021-03-08 16:08:49,2021-04-08 16:08:49,EXPIRED
6,7,2021-03-09 07:15:17,2021-03-09 07:15:17,4,1,2021-03-09 07:15:17,2021-04-09 07:15:17,EXPIRED
7,8,2021-09-17 03:37:43,2021-10-18 00:00:15,6,1,2021-09-17 03:37:43,2021-10-17 03:37:43,EXPIRED
8,9,2021-09-20 07:51:20,2021-10-21 00:00:00,182,1,2021-09-20 07:51:20,2021-10-20 07:51:20,EXPIRED
9,10,2021-09-22 06:27:41,2021-10-23 00:00:16,242,1,2021-09-22 06:27:41,2021-10-22 06:27:41,EXPIRED


In [20]:
# Test Data
accountIds = [1,2,3,6,7,46]

transactions_new = transactions_new[transactions_new['account_id'].isin(accountIds)]
subscriptions_new = subscriptions_new[subscriptions_new['account_id'].isin(accountIds)]
earning_accounts = earning_accounts[earning_accounts['account_id'].isin(accountIds)]
cashbacks_new = cashbacks_new[cashbacks_new['account_id'].isin(accountIds)]
airtime_accounts_new = airtime_accounts_new[airtime_accounts_new['account_id'].isin(accountIds)]
utility_accounts_new = utility_accounts_new[utility_accounts_new['account_id'].isin(accountIds)]

transactions_new['id'].tolist()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 23,
 27,
 29,
 30,
 31,
 34,
 35,
 36,
 55,
 57,
 59,
 60,
 61,
 85,
 86,
 95,
 101,
 103,
 113,
 114,
 120,
 122,
 123,
 126,
 127,
 128,
 132,
 140,
 142,
 145,
 149,
 153,
 160,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 174,
 187,
 194,
 195,
 196,
 197,
 198,
 201,
 207,
 213,
 215,
 237,
 243,
 259,
 265,
 267,
 271,
 279,
 284,
 291,
 298,
 300,
 302,
 308,
 316,
 320,
 322,
 324,
 327,
 328,
 329,
 331,
 332,
 333,
 335,
 362,
 363,
 395,
 433,
 436,
 438,
 440,
 441,
 442,
 475,
 542,
 543,
 544,
 555,
 563,
 574,
 601,
 602,
 634,
 661,
 662,
 665,
 685,
 687,
 732,
 734,
 745,
 751,
 772,
 795,
 827,
 859,
 864,
 870,
 913,
 923,
 942,
 958,
 975,
 985,
 1042,
 1087,
 1151,
 1178,
 1180,
 1236,
 1242,
 1310,
 1433,
 1450,
 1522,
 1599,
 1609,
 1668,
 1696,
 1697,
 1699,
 1777,
 1794,
 1831,
 1837,
 1840,
 1900,
 1977,
 2103,
 2136,
 2232,
 2245,
 2297,
 2323,
 2359,
 2415,
 2416,
 2417,
 2419

In [21]:
transactions_new.to_csv('new/products/sidooh_transactions.csv', index = False)
# at_airtime_requests_new.to_csv('new/products/sidooh_at_airtime_requests.csv', index = False)
# at_airtime_responses_new.to_csv('new/products/sidooh_at_airtime_responses.csv', index = False)
cashbacks_new.to_csv('new/products/sidooh_cashbacks.csv', index = False)

airtime_accounts_new.to_csv('new/products/sidooh_airtime_accounts.csv', index = False)
utility_accounts_new.to_csv('new/products/sidooh_utility_accounts.csv', index = False)

earning_accounts.to_csv('new/products/sidooh_earning_accounts.csv', index = False)

subscriptions_new.to_csv('new/products/sidooh_subscriptions.csv', index = False)